<a href="https://colab.research.google.com/github/sljm12/machine_learning_notebooks/blob/master/LLM/Transcript_Q%26A_with_Gradio_and_VTTSplitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Youtube Transcript Q&A

Using the national day rally speech from https://www.youtube.com/watch?v=LCcWWbx6pXU

Prototype of a Youtube video Q&A with the ability to show the video segments in which the data is taken from from.

Youtube Subtitle Download script from https://github.com/ithelpTheDoctor/Code-Examples/blob/main/Youtube%20Subtitle%20Downloader/subtitles_downloader.py

In [1]:
!pip install yt-dlp auto-gptq optimum huggingface_hub sentencepiece chromadb langchain pypdf sentence_transformers tqdm gradio>/dev/null

In [2]:
!pip install git+https://github.com/sljm12/langchain_vttsplitter >/dev/null

  Running command git clone --filter=blob:none --quiet https://github.com/sljm12/langchain_vttsplitter /tmp/pip-req-build-h02kanm9


# Loading the Model

## Loading the model and tokenizer

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, TextGenerationPipeline

In [4]:
model_name_or_path = "TheBloke/Mistral-7B-OpenOrca-GPTQ" # @param ["TheBloke/zephyr-7B-alpha-GPTQ","TheBloke/Mistral-7B-OpenOrca-GPTQ", "TheBloke/vicuna-13B-v1.5-16K-GPTQ"]

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
#model_name_or_path = "TheBloke/zephyr-7B-alpha-GPTQ"
#model_name_or_path = "TheBloke/Mistral-7B-OpenOrca-GPTQ"
#model_name_or_path = "TheBloke/vicuna-13B-v1.5-16K-GPTQ"
# To use a different branch, change revision
# For example: revision="gptq-4bit-32g-actorder_True"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Vicuna 7B ~ 5GB
Vicuna 13B 14GB

In [6]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!nvidia-smi

Sat Oct 14 03:27:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    27W /  70W |   5383MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Setting up the text generation pipeline

In [7]:
RETURN_FULL_TEXT = False #If you set this to true it will return the prompt text also
pipeline = TextGenerationPipeline(model=model, tokenizer=tokenizer,max_new_tokens=512, return_full_text=RETURN_FULL_TEXT)

In [8]:
prompt_template="""
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
"""


## VICUNA Prompt

In [9]:
VICUNA_TEMPLATE ="""
A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.

USER:

{prompt}

ASSISTANT:

"""

In [10]:
from langchain.chains.retrieval_qa.prompt import PROMPT, prompt_template

#QA_PROMPT = VICUNA_TEMPLATE.format(prompt=prompt_template)

In [11]:
#print(QA_PROMPT)

## MISTRAL Prompt

In [12]:
MISTRAL_Trismegistus_PROMPT='''USER: {prompt}
ASSISTANT:
'''

#QA_PROMPT=MISTRAL_Trismegistus_PROMPT.format(prompt=prompt_template)

In [13]:
#print(QA_PROMPT)

## Mistral OpenOrca Prompt

In [14]:
CHAT_ML_TEMPLATE = """<|im_start|>system
{system_message}<|im_end|>
<|im_start|>user
{prompt}<|im_end|>
<|im_start|>assistant
"""

system_message= "You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your reasoning step-by-step to be sure you get the right answers!"

In [15]:
#QA_PROMPT = CHAT_ML_TEMPLATE.format(system_message=system_message, prompt=prompt_template)

## Zephyr Prompt

In [16]:
ZEPHYR_PRMOPT="""<|system|>
You are a friendly chatbot who always responds in the style of a pirate.</s>
<|user|>
{prompt}</s>
<|assistant|>
"""

In [17]:
#QA_PROMPT=ZEPHYR_PRMOPT.format(prompt=prompt_template)

## Generate prompt

In [18]:
match model_name_or_path:
  case "TheBloke/zephyr-7B-alpha-GPTQ":
    QA_PROMPT=ZEPHYR_PRMOPT.format(prompt=prompt_template)
  case "TheBloke/Mistral-7B-OpenOrca-GPTQ":
    QA_PROMPT = CHAT_ML_TEMPLATE.format(system_message=system_message, prompt=prompt_template)
  case "TheBloke/vicuna-13B-v1.5-16K-GPTQ":
    QA_PROMPT = VICUNA_TEMPLATE.format(prompt=prompt_template)


In [19]:
print(QA_PROMPT)

<|im_start|>system
You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your reasoning step-by-step to be sure you get the right answers!<|im_end|>
<|im_start|>user
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:<|im_end|>
<|im_start|>assistant



# Setting up ChromaDB imports and sentence embeddings

In [20]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader,TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

In [21]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [22]:
# Youtube player iframe
iframe_yt = """
<tr>
<td>
<iframe id="ytplayer" type="text/html" width="640" height="360"
  src="{yt_url}&cc_load_policy=1"
  frameborder="0" ></iframe>
</td>
<td>
{transcript}
</td>
</tr>
"""

In [23]:
table_yt = """
<table>
{rows}
</table>
"""

In [25]:
# The chromaDB variable we put it here so that even when the gradio app stops we can continue to query the DB
db = None

# The Gradio app

In [ ]:
import gradio as gr, os, glob, subprocess,time, datetime,urllib.parse,requests
from langchain.schema.document import Document
from langchain_vttsplitter.vttsplitter import VTTSplitter
from langchain_vttsplitter.loader import YoutubeSubtitleLoader


def query(question):
  global db
  ans=db.similarity_search_with_score(question)
  context="\n".join([i[0].page_content+'\n' for i in ans])
  return pipeline(QA_PROMPT.format(question=question, context=context)), ans

def load_db(url):
  global db
  if db is not None:
    db.delete_collection()

  msg = ""
  try:
    vtt_doc = YoutubeSubtitleLoader(youtube_url=url,language='en').load()
    docs = VTTSplitter().split_text_docs(vtt_doc[0])
    db = Chroma.from_documents(docs, embeddings,collection_metadata={"url":url})
    done_fmt = "Done: {url} {date} Collection Count: {collection_count}"
    msg = done_fmt.format(url=url,
                          date=datetime.datetime.now().isoformat()
                          ,collection_count=str(db._collection.count()))

  except ValueError:
    msg = "Error loading "+url

  return msg

def download(u):
  global db
  return load_db(u)

def extract_embed_url(link,start_time,end_time):
  qs=urllib.parse.urlparse(link).query
  params=urllib.parse.parse_qs(qs)
  return "https://youtube.com/embed/{id}?start={start}&end={end}".format(id=params["v"][0],
                                                                         start=int(start_time),
                                                                         end=int(end_time))

def return_html_iframe(links, transcripts):
  rows = "\n".join([iframe_yt.format(yt_url=l,transcript=transcripts[i]) for i,l in enumerate(links)])
  return table_yt.format(rows=rows)

def query_click(query):
  global db
  ans=db.similarity_search_with_score(query)
  context="\n".join([i[0].page_content+'\n' for i in ans])
  transcripts = [i[0].page_content for i in ans]
  video_ids = [extract_embed_url(i[0].metadata["url"],i[0].metadata["start_time"],i[0].metadata["end_time"]) for i in ans]
  return pipeline(QA_PROMPT.format(question=query, context=context))[0]["generated_text"],return_html_iframe(video_ids, transcripts)

with gr.Blocks() as demo:
  with gr.Row():
    youtube=gr.Textbox(label="Youtube URL", value ="https://www.youtube.com/watch?v=LCcWWbx6pXU", scale=3)
    load_youtube=gr.Button(value="Load Youtube", scale=1)


  youtube_output=gr.Textbox(label="Loading")
  load_youtube.click(download,inputs=[youtube],outputs=[youtube_output])

  with gr.Row():
    query_text=gr.Textbox(label="Query",value="What are the initiatives in the rally?",scale=3)
    query_btn=gr.Button(value="Submit Query", scale=1)


  output=gr.Textbox(lines=7,label="Output")

  html_1=gr.HTML()
  query_btn.click(query_click,inputs=[query_text],outputs=[output,html_1])

demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://f87b573b61b2b9f1f3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


[youtube] Extracting URL: https://www.youtube.com/watch?v=LCcWWbx6pXU
[youtube] LCcWWbx6pXU: Downloading webpage
[youtube] LCcWWbx6pXU: Downloading ios player API JSON
[youtube] LCcWWbx6pXU: Downloading android player API JSON
[youtube] LCcWWbx6pXU: Downloading m3u8 information
[info] LCcWWbx6pXU: Downloading subtitles: en


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
